## Motion Planning through KOMO ##

In [6]:
import robotic as ry
import numpy as np
import time
import os
os.environ['DISPLAY'] = ':0'

## Part a) Adding a constraint to simulation

In [8]:
C = ry.Config()
C.addFile(ry.raiPath('scenarios/pandaSingle.g'))
C.addFrame('way1'). setShape(ry.ST.marker, [.1]) .setPosition([.4, .2, 1.])
C.addFrame('way3'). setShape(ry.ST.marker, [.1]) .setPosition([-.4, .2, 1.])

C.addFrame('box') \
    .setPosition([-.25,.1,1.]) \
    .setShape(ry.ST.ssBox, size=[.06,.6,.6,.005]) \
    .setColor([1,.5,0]) \
    .setContact(True)

C.view()
qHome = C.getJointState()

First, solve the task without collision constraint

In [ ]:
C.setJointState(qHome)
#(config, something with frames, robot speed,?,?)
komo = ry.KOMO(C, 5, 10, 1, True)
komo.addControlObjective([], 0, 1e-1)
komo.addControlObjective([], 1, 1e0)
komo.addObjective([1], ry.FS.positionDiff, ['l_gripper', 'way1'], ry.OT.eq, [1e1])
komo.addObjective([3], ry.FS.positionDiff, ['l_gripper', 'way3'], ry.OT.eq, [1e1])
komo.addObjective([5], ry.FS.positionDiff, ['l_gripper', 'way1'], ry.OT.eq, [1e1])

ret = ry.NLP_Solver(komo.nlp(), verbose=0 ) .solve()
print(ret)
q = komo.getPath()
print(q)

for t in range(q.shape[0]):
    C.setJointState(q[t])
    C.view(False, f'waypoint {t}')
    time.sleep(0.1)

In [18]:
del komo
del C

Then, we add collision constraint with "accumulatedCollisions" feature

In [25]:
C = ry.Config()
C.addFile(ry.raiPath('scenarios/pandaSingle.g')) #robot arm
#                                          size                 x  y   z
C.addFrame('way1'). setShape(ry.ST.marker, [.1]) .setPosition([.4, .2, 1.])
C.addFrame('way3'). setShape(ry.ST.marker, [.1]) .setPosition([-.4, .2, 1.])

C.addFrame('box') \
    .setPosition([-.25,.1,1.]) \
    .setShape(ry.ST.ssBox, size=[.06,.6,.6,.005]) \
    .setColor([1,.5,0]) \
    .setContact(True)

C.view()
qHome = C.getJointState()

But we see that there is a jump in the motion while passing the obstacle

Here we will see the importance of the scale of the constraint (give values 0, 100 and 1 (which is default one) relatively). Then put 0.1 (1e-1) for perfect case. Do not forget to Del K and C or restart.

In [ ]:
C.setJointState(qHome)
komo = ry.KOMO(C, 5, 10, 1, True)
komo.addControlObjective([], 0, 1e-1)
komo.addControlObjective([], 1, 1e0)
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq,[1e-1]) #difference is here
komo.addObjective([1], ry.FS.positionDiff, ['l_gripper', 'way1'], ry.OT.eq, [1e1])
komo.addObjective([3], ry.FS.positionDiff, ['l_gripper', 'way3'], ry.OT.eq, [1e1])
komo.addObjective([5], ry.FS.positionDiff, ['l_gripper', 'way1'], ry.OT.eq, [1e1])

ret = ry.NLP_Solver(komo.nlp(), verbose=0 ) .solve()
print(ret)
q = komo.getPath()
print(q)

for t in range(q.shape[0]):
    C.setJointState(q[t])
    C.view(False, f'waypoint {t}')
    time.sleep(0.1)

komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq);

    komo.addObjective(...): This is calling a method named addObjective on the komo object, which is an instance of a KOMO optimizer.

    []: The first argument [] specifies the time intervals over which this objective applies. In this case, [] means it applies to all time intervals. In other words, this objective is valid for the entire duration of the optimization.

    ry.FS.accumulatedCollisions: The second argument ry.FS.accumulatedCollisions is specifying the type of feature to use for this objective. In this case, it's using a feature related to accumulated collisions. This means the objective will aim to minimize the total amount of collisions that occur throughout the motion.

    []: The third argument [] specifies the frames associated with this feature. Since ry.FS.accumulatedCollisions doesn't require specific frame associations, this argument is left empty.

    ry.OT.eq: The fourth argument ry.OT.eq specifies the type of objective. Here, it's an equality constraint. This means the optimizer will aim to make the value of the accumulated collisions equal to a certain target value (which is default value zero in here).

Putting it all together, this line is instructing the optimizer to minimize the total amount of accumulated collisions over the entire duration of the motion, treating it as an equality constraint. This is a way to prioritize collision-free or low-collision solutions.

##  Part b) Possible State Solution by KOMO

In [30]:
del komo
del C

In [31]:
C = ry.Config()
C.addFile(ry.raiPath('scenarios/pandaSingle.g'))
C.addFrame('way3'). setShape(ry.ST.marker, [.1]) .setPosition([-.4, .2, 1.0])

C.addFrame('box') \
    .setPosition([-.25,.1,1.3]) \
    .setShape(ry.ST.ssBox, size=[.06,.7,.5,.005]) \
    .setColor([1,.5,0]) \
    .setContact(True)

C.view()
qHome = C.getJointState()

In [32]:
C.setJointState(qHome)
komo = ry.KOMO(C, 1, 10, 2, True)
komo.addControlObjective([], 0, 1e-1) 
komo.addControlObjective([], 2, 1e0)
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq,[1e1])
komo.addObjective([1], ry.FS.positionDiff, ['l_gripper', 'way3'], ry.OT.eq, [1e1])
komo.addObjective([1], ry.FS.jointState, [], ry.OT.eq, [1e1], [], order=1)

ret = ry.NLP_Solver(komo.nlp(), verbose=0 ) .solve()
print(ret)
q = komo.getPath()
print('size of path:', q.shape)

for t in range(q.shape[0]):
    C.setJointState(q[t])
    C.view(False, f'waypoint {t}')
    time.sleep(.1)

{ time: 0.4375, evals: 536, done: 1, feasible: 1, sos: 61.119, f: 0, ineq: 0, eq: 0.124065 }
size of path: (10, 7)


## Possible State Solution by RRT

In [33]:
# that's the goal configuration
qT = komo.getPath()[-1]
C.setJointState(qT)
C.view(False, "IK solution")

0

In [34]:
#define a path finding problem
rrt = ry.PathFinder()
rrt.setProblem(C, [qHome], [qT])

ret = rrt.solve()
print(ret)
path = ret.x

{ time: 0.015625, evals: 55, done: 1, feasible: 1, sos: -1, f: -1, ineq: -1, eq: -1 }
-- RRT_PathFinder.cpp:RRT_PathFinder:258(0) initializing with infeasible qT:
query: h_goal: 0 g_coll: 0 isGoal: 1 isFeasible: 0
proxy:  (box)-(l_panda_coll5) [64,51] 	d=-0


In [35]:
# display the path
for t in range(0, path.shape[0]-1):
    C.setJointState(path[t])
    C.view()
    time.sleep(.1)

In [36]:
del komo
del rrt
del C

Now, let us give a state which is impossible for robot arm to reach.

##  Part c) Impossible State Solution by KOMO

In [37]:
C = ry.Config()
C.addFile(ry.raiPath('scenarios/pandaSingle.g'))
C.addFrame('way3'). setShape(ry.ST.marker, [.1]) .setPosition([-.4, .2, 1.])

C.addFrame('box') \
    .setPosition([-.25,.1,1.]) \
    .setShape(ry.ST.ssBox, size=[.06,.7,.7,.005]) \
    .setColor([1,.5,0]) \
    .setContact(True)

C.view()
qHome = C.getJointState()

In [38]:
C.setJointState(qHome)
komo = ry.KOMO(C, 1, 10, 2, True)
komo.addControlObjective([], 0, 1e-1) 
komo.addControlObjective([], 2, 1e0)
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq,[1e1])
komo.addObjective([1], ry.FS.positionDiff, ['l_gripper', 'way3'], ry.OT.eq, [1e1])
komo.addObjective([1], ry.FS.jointState, [], ry.OT.eq, [1e1], [], order=1)

ret = ry.NLP_Solver(komo.nlp(), verbose=0 ) .solve()
print(ret)
q = komo.getPath()
print('size of path:', q.shape)
for t in range(q.shape[0]):
    C.setJointState(q[t])
    C.view(False, f'waypoint {t}')
    time.sleep(.1)

{ time: 0.328125, evals: 264, done: 1, feasible: 0, sos: 28.944, f: 0, ineq: 0, eq: 2.87728 }
size of path: (10, 7)


## Impossible State Solution by RRT

In [39]:
# that's the goal configuration
qT = komo.getPath()[-1]
C.setJointState(qT)
C.view(False, "IK solution")

0

In [40]:
#define a path finding problem
rrt = ry.PathFinder()
rrt.setProblem(C, [qHome], [qT])

ret = rrt.solve()
print(ret)
path = ret.x

-- RRT_PathFinder.cpp:RRT_PathFinder:257(0) initializing with infeasible q0:
query: h_goal: 0 g_coll: 0 isGoal: 1 isFeasible: 0
proxy:  (box)-(l_finger1) [64,58] 	d=-0
proxy:  (box)-(table) [64,1] 	d=-0
proxy:  (box)-(panda_collCameraWrist) [64,62] 	d=-0
proxy:  (box)-(l_finger2) [64,59] 	d=-0
-- RRT_PathFinder.cpp:RRT_PathFinder:258(0) initializing with infeasible qT:
query: h_goal: 0 g_coll: 0 isGoal: 1 isFeasible: 0
proxy:  (box)-(l_finger1) [64,58] 	d=-0
proxy:  (box)-(table) [64,1] 	d=-0
proxy:  (box)-(panda_collCameraWrist) [64,62] 	d=-0
proxy:  (box)-(l_finger2) [64,59] 	d=-0
{ time: 0.359375, evals: 5001, done: 1, feasible: 0, sos: -1, f: -1, ineq: -1, eq: -1 }


So we will get an error since there is no feasible solution

In [41]:
# display the path
for t in range(0, path.shape[0]-1):
    C.setJointState(path[t])
    C.view()
    time.sleep(.1)

IndexError: tuple index out of range

X connection to :0 broken (explicit kill or server shutdown).


In [41]:
# https://marctoussaint.github.io/robotics-course/tutorials.html -> Tutorials
# https://github.com/MarcToussaint/rai-tutorials/tree/main -> Tutorials
# https://marctoussaint.github.io/robotics-course/rai.html -> Documentation